In [3]:
# 즉시 실행 : 그래프를 생성하지 않고 함수를 바로 실행하는 명령형 프로그래밍 환경
# 직관적인 인터페이스 - 코드를 자연스럽게 구조화하고 파이썬의 데이터 구조를 활용, 작은 모델과 작은 데이터를 빠르게 반복
# 손쉬운 디버깅 - 실행중인 모델의 검토나 변경 사항의 테스트를 위해서 연산을 직접 호출, 에러 확인을 위한 표준 파이썬 디버깅 툴 사용
# 자연스러운 흐름 제어 - 그래프 제어 흐름 대신 파이썬 제어 흐름을 사용함으로써 동적인 모델 구조의 단순화


# 설치와 기본 사용법
import tensorflow as tf
import cProfile

# tensorflow 2.0에서 즉시 실행은 기본적으로 활성화
# tf.executing_eagerly()

# tensorflow 연산을 바로 실행할 수 있고, 즉시 결과 확인 가능
# x = [[2.]]
# m = tf.matmul(x, x)
# print("hello, {}".format(m))

# a = tf.constant([[1, 2],
#                  [3, 4]])
# print(a)

# 브로드캐스팅 지원
# b = tf.add(a, 1)
# print(b)

# 연산자 오버로딩 지원
# print(a * b)

# NumPy 값 사용
import numpy as np

# c = np.multiply(a, b)
# print(c)

# 텐서로부터 numpy값 얻기
# print(a.numpy())


# 동적인 제어 흐름
# def fizzbuzz(max_num):
#     counter = tf.constant(0)
#     max_num = tf.convert_to_tensor(max_num)
#     for num in range(1, max_num.numpy() + 1):
#         num = tf.constant(num)
#         if int(num % 3) == 0 and int(num % 5) == 0:
#             print("FizzBuzz")
#         elif int(num % 3) == 0:
#             print("Fizz")
#         elif int(num % 5) == 0:
#             print("Buzz")
#         else:
#             print(num.numpy())
#         counter+=1

# fizzbuzz(15)


# 즉시 훈련
## 그래디언트 계산 (tf.GradientTape 사용)
# w = tf.Variable([[1.0]])
# with tf.GradientTape() as tape:
#     loss = w * w

# grad = tape.gradient(loss, w)
# print(grad)

## 모델 훈련
# mnist 데이터 로드 및 포맷 설정
# (mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

# dataset = tf.data.Dataset.from_tensor_slices(
#     (tf.cast(mnist_images[..., tf.newaxis] / 255, tf.float32),
#      tf.cast(mnist_labels, tf.int64)))
# dataset = dataset.shuffle(1000).batch(32)

# 모델 생성
# mnist_model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, [3, 3], activation='relu',
#                            input_shape=(None, None, 1)),
#     tf.keras.layers.Conv2D(16, [3, 3], activation='relu'),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(10)
# ])

# 즉시 실행에서는 훈련없이도 모델을 사용하고 결과를 점검할 수 있음
# for images, labels in dataset.take(1):
#     print("Logit: ", mnist_model(images[0:1]).numpy())

# optimizer = tf.keras.optimizers.Adam()
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# loss_history = []

# def train_step(images, labels):
#     with tf.GradientTape() as tape:
#         logits = mnist_model(images, training=True)

#         # 결과의 형태를 확인하기 위해 단언문 추가
#         tf.debugging.assert_equal(logits.shape, (32, 10))

#         loss_value = loss_object(labels, logits)

#     loss_history.append(loss_value.numpy().mean())
#     grads = tape.gradient(loss_value, mnist_model.trainable_variables)
#     optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

# def train():
#     for epoch in range(3):
#         for (batch, (images, labels)) in enumerate(dataset):
#             train_step(images, labels)
#         print("Epoch {} 종료".format(epoch))

# train()

# import matplotlib.pyplot as plt

# plt.plot(loss_history)
# plt.xlabel("Batch #")
# plt.ylabel("Loss [entropy]")

## 변수와 옵티마이저
# class Model(tf.keras.Model):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.W = tf.Variable(5., name='weight')
#         self.B = tf.Variable(10., name='bias')

#     def call(self, inputs):
#         return inputs * self.W + self.B

# 약 3 * x + 2개의 점으로 구성된 실험 데이터
# NUM_EXAMPLES = 2000
# training_inputs = tf.random.normal([NUM_EXAMPLES])
# noise = tf.random.normal([NUM_EXAMPLES])
# training_outputs = training_inputs * 3 + 2 + noise

# 최적화할 손실 함수
# def loss(model, inputs, targets):
#     error = model(inputs) - targets
#     return tf.reduce_mean(tf.square(error))

# def grad(model, inputs, targets):
#     with tf.GradientTape() as tape:
#         loss_value = loss(model, inputs, targets)
#     return tape.gradient(loss_value, [model.W, model.B])

# 정의 :
# 1. 모델
# 2. 모델 파라미터에 대한 손실 함수의 미분
# 3. 미분에 기초한 변수 업데이트 전략
# model = Model()
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# print("초기 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# 반복 훈련
# for i in range(300):
#     grads = grad(model, training_inputs, training_outputs)
#     optimizer.apply_gradients(zip(grads, [model.W, model.B]))
#     if i % 20 == 0:
#         print("Step {:03d}에서 손실: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

# print("최종 손실: {:3f}".format(loss(model, training_inputs, training_outputs)))
# print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))


# 즉시 실행에서 상태를 위한 객체 사용
## 변수는 객체
# if tf.config.experimental.list_physical_devices("GPU"):
#     with tf.device("gpu:0"):
#         print("GPU 사용 가능")
#         v = tf.Variable(tf.random.normal([1000, 1000]))
        # v = None                                # v는 더 이상 GPU 메모리를 사용하지 않음

## 객체 기반 저장
# x = tf.Variable(10.)
# checkpoint = tf.train.Checkpoint(x=x)

# x.assigh(2.)                                    # 변수에 새로운 값을 할당하고 저장
# checkpoint_path = "./ckpt/"
# checkpoint.save("./ckpt/")

# x.assgin(11.)                                   # 저장한 후 변수 변경

# 체크포인트로부터 값 복구
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

# print(x)

# import os

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(16, [3, 3], activation='relu'),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(10)
# ])
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# checkpoint_dir = "path/to/model_dir"
# if not os.path.exists(checkpoint_dir):
#     os.makedirs(checkpoint_dir)

# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# root = tf.train.Checkpoint(optimizer=optimizer,
#                            model=model)
# root.save(checkpoint_prefix)
# root.restore(tf.train.latest_checkpoint(checkpoint_dir))

## 객체 지향형 지표
# m = tf.keras.metrics.Mean("loss")
# m(0)
# m(5)
# m.result()
# m([8, 9])
# m.result()


# summary와 TensorBoard
# TensorBoard : 훈련 과정에서 모델을 파악하거나 디버깅하고 최적화하기 위해 사용하는 시각화 도구
# TensorBoard는 프로그램이 실행되는 동안 작성된 summary 이벤트를 사용
# logdir = "./tb/"
# writer = tf.summary.create_file_writer(logdir)

# with writer.as_default():                       # 또는 반복 전 writer.set_as_default()를 호출
#     for i in range(1000):
#         step = i + 1
#         # 실제 훈련 함수로 손실 계산
#         loss = 1 - 0.001 * step
#         if step % 100 == 0:
#             tf.summary.scalar('손실', loss, step=step)

# !ls tb/

# 자동 미분 관련 고급편
## 동적 모델
# def line_search_step(fn, init_x, rate=1.0):
#     with tf.GradientTape() as tape:
#         # 변수는 자동적으로 기록되지만 텐서는 사용자 스스로 확인해야 함
#         tape.watch(init_x)
#         value = fn(init_x)

#     grad = tape.gradient(value, init_x)
#     grad_norm = tf.reduce_sum(grad * grad)
#     init_value = value
#     while value > init_value - rate * grad_norm:
#         x = init_x - rate * grad
#         value = fn(x)
#         rate /= 2.0
#     return x, value

## 사용자 정의 그래디언트
# 정방향 함수 안에서 입력값 또는 출력값, 중간값과 관련된 그래디언트를 정의해야 함
# @tf.custom_gradient
# def clip_gradient_by_norm(x, norm):
#     y = tf.identify(x)
#     def grad_fn(dresult):
#         return [tf.clip_by_norm(dresult, norm), None]
#     return y, grad_fn

# 사용자 정의 그래디언트는 일반적으로 연산에 대해 수치적으로 안정된 그래디언트를 제공하기 위해 사용
# def log1pexp(x):
#     return tf.math.log(1 + tf.exp(x))

# def grad_log1pexp(x):
#     with tf.GradientTape() as tape:
#         tape.watch(x)
#         value = log1pexp(x)
#     return tape.gradient(value, x)

# 그래디언트 계산은 x = 0일 때 잘 동작함
# grad_log1pexp(tf.constant(0.)).numpy()

# 그러나, x = 100일 때 수치적으로 불안정하기 때문에 실패
# grad_log1pexp(tf.constant(100.)).numpy()

# 불필요한 계산을 제거함으로써 계산을 효율적으로 하기 위해 정방향 경로 안에서 계산된 tf.exp(x) 값을 재사용
# @tf.custom_gradient
# def log1pexp(x):
#     e = tf.exp(x)
#     def grad(dy):
#         return dy * (1 - 1 / (1 + e))
#     return tf.math.log(1 + e), grad

# def grad_log1pexp(x):
#     with tf.GradientTape() as tape:
#         tape.watch(x)
#         value = log1pexp(x)
#     return tape.gradient(value, x)

# 전처럼, 그래디언트 계산은 x = 0일 때 잘 동작
# grad_log1pexp(tf.constant(0.)).numpy()

# 그래디언트 계산은 x = 100일 때 역시 잘 동작
# grad_log1pexp(tf.constant(100.)).numpy()


# 성능
import time

def measure(x, steps):
    # 텐서플로는 처음 사용할 때 GPU를 초기화, 시간 계산에서 제외
    tf.matmul(x, x)
    start = time.time()
    for i in range(steps):
        x = tf.matmul(x, x)

    # tf.matmul는 행렬 곱셉을 완료하기 전에 결과 반환 가능
    # 아래 x.numpy() 호출은 대기열에 추가된 모든 연산이 완료된 것임을 보장
    # (그리고 그 결과가 호스트 메모리에 복사되므로 matmul 연산 시간은 조금 많은 연산 시간이 포함됨)

    _ = x.numpy()
    end = time.time()
    return end - start

shape = (1000, 1000)
steps = 200
print("{} 크기 행렬을 자기 자신과 {}번 곱했을 때 걸리는 시간:".format(shape, steps))

# CPU에서 실행
with tf.device("/cpu:0"):
    print("CPU: {} 초".format(measure(tf.random.normal(shape), steps)))

# GPU에서 실행(가능하다면)
if tf.config.experimental.list_physical_devices("GPU"):
    with tf.device("/gpu:0"):
        print("GPU: {} 초".format(measure(tf.random.normal(shape), steps)))
else:
    print("GPU 없음")

if tf.config.experimental.list_physical_devices("GPU"):
    x = tf.random.normal([10, 10])

    x_gpu0 = x.gpu()
    x_cpu = x.cpu()

    _ = tf.matmul(x_cpu, x_cpu)                 # CPU에서 실행
    _ = tf.matmul(x_gpu0, x_gpu0)               # GPU:0에서 실행

(1000, 1000) 크기 행렬을 자기 자신과 200번 곱했을 때 걸리는 시간:
CPU: 8.204607725143433 초
GPU 없음
